# Challenge five: Aero Alerts
Automated weather alerts for airports

**Goal: Demonstrate your ability to build an automated end-to-end service integrating BigQuery, Gemini, and Looker Studio**

## Requirements:
* Download and process data using Cloud Storage and BigQuery
* Program an external API to retrieve current information
* Use Gemin to create alerts that combine the BigQuery and external data
* Build a Looker Studio dashboard that provides user access to the alerts
* Create an automated GenAIOps pipeline using Apache Airflow and Google Cloud Compose

## Instructions


1. Create a Colab Enterprise Notebook for all your code.
2. Download the following CSV data into Cloud Storage and then load it into BigQuery:
`gs://labs.roitraining.com/data-to-ai-workshop/airports.csv`
3. Use the National Weather Service API at the the following URL to get the weather forecast for all the
large airports in the US (for this proof of concept only use large airports).
a. https://api.weather.gov
4. Use Gemini to create an alert for each airport and store the alerts in BigQuery.
5. Create a Looker Studio dashboard that provides the data to users.
6. Automate the data process steps using a Google Cloud Composer or Kubeflow pipeline.
7. Bonus: Display a map of the US on your Looker Studio dashboard. Allow users to select an airport
from the map to see the alert for that location.
8. Upload your Notebook to your GitHub repository for grading.

Download the following CSV data into Cloud Storage and then load it into BigQuery: gs://labs.roitraining.com/data-to-ai-workshop/airports.csv

In [1]:
%%bigquery
CREATE SCHEMA IF NOT EXISTS challenge_5
OPTIONS(
location="us",
default_table_expiration_days=14 );

Query is running:   0%|          |

""


In [2]:
%%bigquery
CREATE OR REPLACE EXTERNAL TABLE challenge_5.airports
OPTIONS (
  format = 'CSV',
  uris = ['gs://labs.roitraining.com/data-to-ai-workshop/airports.csv'],
  skip_leading_rows = 1,
  allow_jagged_rows = false,
  allow_quoted_newlines = false,
  field_delimiter = ',',
  max_bad_records = 10
)


Query is running:   0%|          |

""


Use the National Weather Service API at the the following URL to get the weather forecast for all the large airports in the US (for this proof of concept only use large airports). `https://api.weather.gov`. Then use Gemini to create an alert for each airport and store the alerts in a new BigQuery table.

In [1]:
import pandas as pd
import vertexai
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel

# --- 1. Configuration ---
# IMPORTANT: Replace with your Google Cloud project ID and a valid region.
LOCATION = "us-central1"  # e.g., "us-central1"
DATASET_ID = "challenge_5"
SOURCE_TABLE = "airports"
DESTINATION_TABLE = "airports_with_forecasts_and_alerts"

# Construct the full table IDs
source_table_id = f"{DATASET_ID}.{SOURCE_TABLE}"
dest_table_id = f"{DATASET_ID}.{DESTINATION_TABLE}"

# --- 2. Initialize Clients and Models ---
# Initialize the BigQuery client
client = bigquery.Client()

# Initialize Vertex AI SDK
# In a Vertex AI notebook, authentication is handled automatically.
vertexai.init(location=LOCATION)

# Load the Gemini Pro model
model = GenerativeModel("gemini-2.0-flash")
print("Vertex AI and Gemini model initialized.")

# --- 3. Helper Functions ---

# Doug's 'get_extended_weather_forecast' function.
# @title


import requests
from typing import Optional, List, Dict


def get_extended_weather_forecast(lat: float, lon: float) -> Optional[List[Dict[str, str]]]:
   """
   Fetch the extended weather forecast from the U.S. National Weather Service (NWS) API
   based on a given latitude and longitude.


   Args:
       lat (float): Latitude of the location (e.g., 38.8977).
       lon (float): Longitude of the location (e.g., -77.0365).


   Returns:
       Optional[List[Dict[str, str]]]: A list of forecast dictionaries for each time period,
       each containing:
           - 'name': Name of the forecast period (e.g., "Today", "Tonight")
           - 'startTime': ISO timestamp for the start of the forecast period
           - 'temperature': Temperature value
           - 'temperatureUnit': Temperature unit (e.g., "F" or "C")
           - 'windSpeed': Wind speed description
           - 'windDirection': Wind direction (e.g., "NW")
           - 'shortForecast': Short summary (e.g., "Partly Sunny")
           - 'detailedForecast': Full text forecast


       Returns None if data is unavailable or an error occurs.
   """
   headers = {
       'User-Agent': 'MyWeatherApp (doug@roitraining.com)',  # Replace with your actual email
       'Accept': 'application/geo+json'
   }


   # Step 1: Get metadata to find forecast URL
   points_url = f"https://api.weather.gov/points/{lat},{lon}"
   response = requests.get(points_url, headers=headers)


   if response.status_code != 200:
       print(f"Error fetching data from points endpoint: {response.status_code}")
       return None


   points_data = response.json()
   forecast_url = points_data['properties'].get('forecast')
   if not forecast_url:
       print("Forecast URL not found in response.")
       return None


   # Step 2: Fetch the forecast data
   forecast_response = requests.get(forecast_url, headers=headers)
   if forecast_response.status_code != 200:
       print(f"Error fetching forecast: {forecast_response.status_code}")
       return None


   forecast_data = forecast_response.json()
   periods = forecast_data['properties'].get('periods', [])


   if not periods:
       print("No forecast periods found in response.")
       return None


   # Return the full extended forecast
   extended_forecast = []
   for period in periods:
       extended_forecast.append({
           'name': period['name'],
           'startTime': period['startTime'],
           'temperature': str(period['temperature']),
           'temperatureUnit': period['temperatureUnit'],
           'windSpeed': period['windSpeed'],
           'windDirection': period['windDirection'],
           'shortForecast': period['shortForecast'],
           'detailedForecast': period['detailedForecast']
       })


   return extended_forecast

def generate_airport_alert(forecast: dict) -> str:
    """
    Uses the Gemini model to generate a concise operational alert for an airport.

    Args:
        forecast: A dictionary containing forecast details for one time period.

    Returns:
        A string containing the generated alert.
    """
    if not forecast:
        return "No forecast data to generate an alert."

    # Create a clear, structured prompt for the model
    prompt = f"""
    You are an expert aviation operations assistant. Your task is to write a concise, clear operational alert for an airport based on the provided weather forecast.

    The alert should be a single sentence and focus on potential impacts to flights (e.g., delays, cancellations, turbulence, ground stops). Do not use introductory phrases like "The alert is:". Be professional and direct.

    Weather Data:
    - Summary: {forecast.get('shortForecast', 'N/A')}
    - Temperature: {forecast.get('temperature', 'N/A')}°{forecast.get('temperatureUnit', '')}
    - Wind: {forecast.get('windSpeed', 'N/A')} from the {forecast.get('windDirection', 'N/A')}
    - Details: {forecast.get('detailedForecast', 'N/A')}

    Generate the operational alert now.
    """

    try:
        print("   -> Generating alert with Gemini...")
        response = model.generate_content(prompt)
        # Clean up the response text
        return response.text.strip()
    except Exception as e:
        print(f"   -> An error occurred during Gemini API call: {e}")
        return "Alert generation failed."


# --- 4. Fetch Large Airports from BigQuery ---
print(f"\nQuerying large airports from {source_table_id}...")
query = f"SELECT * FROM `{source_table_id}` WHERE type = 'large_airport' AND iso_country = 'US'"

try:
    df = client.query(query).to_dataframe()
    print(f"Successfully fetched {len(df)} large airports.")
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")
    exit()

# --- 5. Process Each Airport, Get Forecast, and Generate Alert ---
processed_rows = []
print("\nProcessing each airport to enrich with weather data and alerts...")

for index, airport_row in df.iterrows():
    print(f"Processing airport: {airport_row['name']}")

    combined_data = airport_row.to_dict()

    forecast_list = get_extended_weather_forecast(
        airport_row['latitude_deg'],
        airport_row['longitude_deg']
    )

    if forecast_list:
        first_forecast = forecast_list[0]
        combined_data.update(first_forecast)

        # Generate the alert using the forecast data
        alert_text = generate_airport_alert(first_forecast)
        combined_data['airport_alert'] = alert_text
    else:
        print(f"   -> No forecast data returned for {airport_row['name']}.")
        # Add placeholder values for all new columns if no forecast is available
        forecast_keys = ['name', 'startTime', 'temperature', 'temperatureUnit', 'windSpeed', 'windDirection', 'shortForecast', 'detailedForecast', 'airport_alert']
        for key in forecast_keys:
            combined_data[key] = None # Use None for missing data
        combined_data['airport_alert'] = "No forecast data available."

    processed_rows.append(combined_data)

enriched_df = pd.DataFrame(processed_rows)
print("\nData processing complete.")
print("Preview of the enriched data with alerts:")
print(enriched_df[['name', 'shortForecast', 'airport_alert']].head())

# --- 6. Upload the Enriched DataFrame to a New BigQuery Table ---
print(f"\nUploading enriched data to new table: {dest_table_id}...")
job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
load_job = client.load_table_from_dataframe(enriched_df, dest_table_id, job_config=job_config)
load_job.result()
print(f"Success! Table '{dest_table_id}' has been created/updated with {len(enriched_df)} rows.")


/usr/local/lib/python3.11/dist-packages/vertexai/generative_models/_generative_models.py:433: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Vertex AI and Gemini model initialized.

Querying large airports from challenge_5.airports...
Successfully fetched 71 large airports.

Processing each airport to enrich with weather data and alerts...
Processing airport: Albuquerque International Sunport
   -> Generating alert with Gemini...
Processing airport: Joint Base Andrews
   -> Generating alert with Gemini...
Processing airport: Hartsfield Jackson Atlanta International Airport
   -> Generating alert with Gemini...
Processing airport: Austin Bergstrom International Airport
   -> Generating alert with Gemini...
Processing airport: Bradley International Airport
   -> Generating alert with Gemini...
Processing airport: Nashville International Airport
   -> Generating alert with Gemini...
Processing airport: Logan International Airport
   -> Generating alert with Gemini...
Processing airport: Buffalo Niagara International Airport
   -> Generating alert with Gemini...
Processing airport: Baltimore/Washington International Thurgood Ma